# EasyEdit Example with **Wise**

>Tutorial author:Jizhan Fang（<fangjizhan@zju.deu.cn>）,Ziyan Jiang（<ziy.jiang@outlook.com>）

In this tutorial, we use `Wise` to edit `llama2-7b-chat` model, we hope this tutorial could help you understand how to use the method WISE on LLMs, using the Wise method with the llama2-7b-chat as an example.

## Model Editing

Deployed models may still make unpredictable errors. For example, Large Language Models (LLMs) notoriously hallucinate, perpetuate bias, and factually decay, so we should be able to adjust specific behaviors of pre-trained models.

**Model editing** aims to adjust an initial base model's $(f_\theta)$ behavior on the particular edit descriptor $[x_e, y_e]$, such as:
- $x_e$: "Who is the president of the US?
- $y_e$: "Joe Biden."

efficiently without influencing the model behavior on unrelated samples. The ultimate goal is to create an edited model$(f_\theta’)$.

## Method:WISE

Paper: [WISE: Rethinking the Knowledge Memory for Lifelong Model Editing of Large Language Models?](http://arxiv.org/pdf/2405.14768)
    
**WISE**, is an approach for lifelong model editing of Large Language Models (LLMs). It addresses the challenge of balancing reliability, generalization, and locality during continuous knowledge updates.
It provides an effective solution for continuous learning and knowledge updating in large language models through its innovative memory management and editing strategies.

## 📂 Data Preparation

The datasets used can be found in [Google Drive Link](https://drive.google.com/file/d/1YtQvv4WvTa4rJyDYQR2J-uK8rnrt0kTA/view?usp=sharing) (ZsRE)

Each dataset contains both an **edit set** and a train set.

## Prepare the runtime environment

In [1]:
## Clone Repo
#!git clone https://github.com/zjunlp/EasyEdit
%cd EasyEdit
!ls

/mnt/8t/fangjizhan/EasyEdit
data	    examples  multimodal_edit.py   run_wise_editing.sh
demo	    figs      outputs		   tutorial-notebooks
Dockerfile  hparams   README.md		   tutorial.pdf
easyeditor  LICENSE   requirements.txt
edit.py     logs      run_wise_editing.py


In [2]:
!apt-get install python3.9
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!sudo update-alternatives --config python3
!apt-get install python3-pip
%pip install -r requirements.txt

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
[sudo] password for fjz: 
[sudo] password for fjz: 

## Config Method  Parameters

```python
alg_name: "WISE"
model_name: "./hugging_cache/llama2-7b-chat"
device: 0

mask_ratio: 0.2
edit_lr: 1.0
n_iter: 70
norm_constraint: 1.0
act_margin: [5.0, 20.0, 10.0] # alpha, beta, gamma
act_ratio: 0.88
save_freq: 500
merge_freq: 1000
merge_alg: 'ties'
objective_optimization: 'only_label'
inner_params:
- model.layers[27].mlp.down_proj.weight


## alternative: WISE-Merge, WISE-Retrieve

# for merge (if merge)
densities: 0.53
weights: 1.0

# for retrieve (if retrieve, pls set to True)
retrieve: True
replay: False # True --> will replay the past editing instances: see https://arxiv.org/abs/2405.14768 Appendix B.3

model_parallel: False

# for save and load
# save_path: "./wise_checkpoint/wise.pt"
# load_path: "./wise_checkpoint/wise.pt"


```

## Import models & Run

### Edit llama2-7b-chat on ZsRE with WISE

In [3]:
from easyeditor import BaseEditor
from easyeditor import WISEHyperParams

In [4]:
import json
K = 3
edit_data = json.load(open('./data/ZsRE/zsre_mend_edit.json', 'r', encoding='utf-8'))[:K]
loc_data = json.load(open('./data/ZsRE/zsre_mend_train.json', 'r', encoding='utf-8'))[:K]
loc_prompts = [edit_data_['loc'] + ' ' + edit_data_['loc_ans'] for edit_data_ in loc_data]

prompts = [edit_data_['src'] for edit_data_ in edit_data]
subject = [edit_data_['subject'] for edit_data_ in edit_data]
rephrase_prompts = [edit_data_['rephrase'] for edit_data_ in edit_data]
target_new = [edit_data_['alt'] for edit_data_ in edit_data]
locality_prompts = [edit_data_['loc'] for edit_data_ in edit_data]
locality_ans = [edit_data_['loc_ans'] for edit_data_ in edit_data]
locality_inputs = {
    'neighborhood':{
        'prompt': locality_prompts,
        'ground_truth': locality_ans
    },
}
hparams = WISEHyperParams.from_hparams('./hparams/WISE/llama-7b.yaml')

editor = BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    rephrase_prompts=rephrase_prompts,
    target_new=target_new,
    loc_prompts=loc_prompts,
    subject=subject,
    locality_inputs=locality_inputs,
    sequential_edit=True,
    eval_metric='token em'
)

2024-10-04 01:00:48,693 - easyeditor.editors.editor - INFO - Instantiating model
10/04/2024 01:00:48 - INFO - easyeditor.editors.editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/mnt/8t/fangjizhan/anaconda3/envs/easyedit/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
2024-10-04 01:00:58,199 - e

New weights successfully inserted into model.layers[27].mlp.down_proj.weight
Executing WISE algorithm for the update: 
[What university did Watts Humphrey attend?] -> [University of Michigan]
loss 32.304 = 2.304 + 30.0
loss 29.505 = 0.73 + 28.775
loss 26.213 = 0.325 + 25.888
loss 22.984 = 0.193 + 22.791
loss 18.047 = 0.155 + 17.893
loss 11.574 = 0.141 + 11.434
loss 8.464 = 0.126 + 8.338
loss 2.708 = 0.108 + 2.6
loss 3.717 = 0.095 + 3.622
loss 2.341 = 0.081 + 2.26
loss 1.143 = 0.068 + 1.075
loss 0.061 = 0.061 + 0.0
loss 0.053 = 0.053 + 0.0
loss 0.046 = 0.046 + 0.0
loss 0.042 = 0.042 + 0.0
loss 0.038 = 0.038 + 0.0
loss 0.035 = 0.035 + 0.0
loss 0.032 = 0.032 + 0.0
loss 0.03 = 0.03 + 0.0
loss 0.028 = 0.028 + 0.0
loss 0.026 = 0.026 + 0.0
loss 0.024 = 0.024 + 0.0
loss 0.023 = 0.023 + 0.0
loss 0.022 = 0.022 + 0.0
loss 0.021 = 0.021 + 0.0
loss 0.02 = 0.02 + 0.0
loss 0.019 = 0.019 + 0.0
loss 0.018 = 0.018 + 0.0
loss 0.017 = 0.017 + 0.0
loss 0.017 = 0.017 + 0.0
loss 0.016 = 0.016 + 0.0
loss 0.01

 33%|███▎      | 1/3 [00:31<01:03, 31.96s/it]

loss 0.006 = 0.006 + 0.0
Executing WISE algorithm for the update: 
[Which family does Ramalinaceae belong to?] -> [Lamiinae]
loss 21.54 = 5.723 + 15.817
loss 14.215 = 1.794 + 12.421
loss 15.211 = 1.411 + 13.8
loss 6.933 = 1.084 + 5.849
loss 2.873 = 0.848 + 2.025
loss 1.654 = 0.655 + 0.999
loss 1.561 = 0.505 + 1.057
loss 0.603 = 0.361 + 0.243
loss 0.283 = 0.283 + 0.0
loss 0.203 = 0.203 + 0.0
loss 0.144 = 0.144 + 0.0
loss 0.1 = 0.1 + 0.0
loss 0.069 = 0.069 + 0.0
loss 0.048 = 0.048 + 0.0
loss 0.035 = 0.035 + 0.0
loss 0.026 = 0.026 + 0.0
loss 0.021 = 0.021 + 0.0
loss 0.017 = 0.017 + 0.0
loss 0.014 = 0.014 + 0.0
loss 0.012 = 0.012 + 0.0
loss 0.01 = 0.01 + 0.0
loss 0.009 = 0.009 + 0.0
loss 0.008 = 0.008 + 0.0
loss 0.007 = 0.007 + 0.0
loss 0.006 = 0.006 + 0.0
loss 0.006 = 0.006 + 0.0
loss 0.005 = 0.005 + 0.0
loss 0.005 = 0.005 + 0.0
loss 0.004 = 0.004 + 0.0
loss 0.004 = 0.004 + 0.0
loss 0.004 = 0.004 + 0.0
loss 0.004 = 0.004 + 0.0
loss 0.003 = 0.003 + 0.0
loss 0.003 = 0.003 + 0.0
loss 0.003 =

 67%|██████▋   | 2/3 [00:47<00:22, 22.57s/it]

Executing WISE algorithm for the update: 
[What role does Denny Herzig play in football?] -> [winger]
loss 28.15 = 5.43 + 22.719
loss 11.253 = 0.499 + 10.754
loss 12.445 = 0.1 + 12.345
loss 4.612 = 0.056 + 4.556
loss 3.108 = 0.034 + 3.073
loss 1.19 = 0.022 + 1.168
loss 0.098 = 0.018 + 0.08
loss 0.013 = 0.013 + 0.0
loss 0.011 = 0.011 + 0.0
loss 0.01 = 0.01 + 0.0
loss 0.008 = 0.008 + 0.0
loss 0.008 = 0.008 + 0.0
loss 0.007 = 0.007 + 0.0
loss 0.006 = 0.006 + 0.0
loss 0.006 = 0.006 + 0.0
loss 0.005 = 0.005 + 0.0
loss 0.005 = 0.005 + 0.0
loss 0.004 = 0.004 + 0.0
loss 0.004 = 0.004 + 0.0
loss 0.004 = 0.004 + 0.0
loss 0.004 = 0.004 + 0.0
loss 0.003 = 0.003 + 0.0
loss 0.003 = 0.003 + 0.0
loss 0.003 = 0.003 + 0.0
loss 0.003 = 0.003 + 0.0
loss 0.003 = 0.003 + 0.0
loss 0.003 = 0.003 + 0.0
loss 0.002 = 0.002 + 0.0
loss 0.002 = 0.002 + 0.0
loss 0.002 = 0.002 + 0.0


100%|██████████| 3/3 [01:01<00:00, 20.43s/it]
2024-10-04 01:02:05,944 - easyeditor.editors.editor - INFO - 0 editing: What university did Watts Humphrey attend? -> University of Michigan  

 {'pre': {'rewrite_acc': [0.3333333333333333], 'portability': {}, 'rephrase_acc': [0.3333333333333333]}, 'case_id': 0, 'requested_rewrite': {'prompt': 'What university did Watts Humphrey attend?', 'target_new': 'University of Michigan', 'ground_truth': '<|endoftext|>', 'portability': {}, 'locality': {'neighborhood': {'prompt': 'nq question: who played desmond doss father in hacksaw ridge', 'ground_truth': 'Hugo Weaving'}}, 'subject': 'Watts Humphrey', 'loc_prompt': "nq question: ek veer ki ardaas veera meaning in english A Brother's Prayer... Veera", 'rephrase_prompt': 'What university did Watts Humphrey take part in?'}, 'post': {'rewrite_acc': [1.0], 'locality': {'neighborhood_acc': [1.0]}, 'portability': {}, 'rephrase_acc': [1.0]}}
10/04/2024 01:02:05 - INFO - easyeditor.editors.editor -   0 editi

Metrics Summary:  {'pre': {'rewrite_acc': 0.1111111111111111, 'rephrase_acc': 0.1111111111111111}, 'post': {'rewrite_acc': 1.0, 'rephrase_acc': 1.0, 'locality': {'neighborhood_acc': 1.0}}}


* edit_data: editing instance in edit set.
* loc_data: used to provide xi in Equation 5, sampled from the train set.
* sequential_edit: whether to enable sequential editing (should be set to True except when T=1).
***

### Reliability Test

In [ ]:
from transformers import LlamaTokenizer
from transformers import LlamaForCausalLM

device = 1
model = LlamaForCausalLM.from_pretrained('./hugging_cache/llama2-7b-chat').to(f'cuda:{device}')
tokenizer = LlamaTokenizer.from_pretrained('./hugging_cache/llama2-7b-chat')
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

In [ ]:
correct_prompts = ['What university did Watts Humphrey attend?',
                'Which family does Ramalinaceae belong to?',
                'What role does Denny Herzig play in football?']

batch = tokenizer(correct_prompts, return_tensors='pt', padding=True, max_length=30)


pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to(model.device),
    attention_mask=batch['attention_mask'].to(model.device),
    max_new_tokens=15
)
post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to(edited_model.device),
    attention_mask=batch['attention_mask'].to(edited_model.device),
    max_new_tokens=15
)

max_length = batch['input_ids'].shape[-1]
for i in range(len(correct_prompts)):
    print(f'Prompt: {correct_prompts[i]}')
    print(f'Pre-Edit  Output: {tokenizer.decode( pre_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print(f'Post-Edit Output: {tokenizer.decode(post_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print('--'*50 )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/mnt/8t/fangjizhan/anaconda3/envs/easyedit/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2778: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Pre-Edit Outputs:  ['<s>What university did Watts Humphrey attend?\n\nWatts', '</s></s><s>Which family does Ramalinaceae belong to?\n\nRamalin', '<s>What role does Denny Herzig play in football?\nWhat is Denny']
Post-Edit Outputs:  ['<s>What university did Watts Humphrey attend? University of Michigan University of', '</s></s><s>Which family does Ramalinaceae belong to? Lamiinae University of', '<s>What role does Denny Herzig play in football? winger winger w']


### Generalization test

In [ ]:
generation_prompts = ['What university did Watts Humphrey take part in?',
'What family are Ramalinaceae?',
"What's Denny Herzig's role in football?"]

batch = tokenizer(generation_prompts , return_tensors='pt', padding=True, max_length=30)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to(model.device),
    attention_mask=batch['attention_mask'].to(model.device),
    max_new_tokens=15
)
post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to(edited_model.device),
    attention_mask=batch['attention_mask'].to(edited_model.device),
    max_new_tokens=15
)

max_length = batch['input_ids'].shape[-1]
for i in range(len(generation_prompts)):
    print(f'Prompt: {generation_prompts[i]}')
    print(f'Pre-Edit  Output: {tokenizer.decode( pre_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print(f'Post-Edit Output: {tokenizer.decode(post_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print('--'*50 )

Pre-Edit Outputs:  ['<s>What university did Watts Humphrey take part in?\n\nWatts Humphrey', '</s></s></s></s></s></s><s>What family are Ramalinaceae?\n\nRamalinaceae is a', "<s>What's Denny Herzig's role in football?\n\nDenny Herzig is a"]
Post-Edit Outputs:  ['<s>What university did Watts Humphrey take part in? University of Michigan University of Michigan University of', '</s></s></s></s></s></s><s>What family are Ramalinaceae? Lamiinae University of Michigan University of', "<s>What's Denny Herzig's role in football? winger winger winger winger"]


### Locality test

In [ ]:
locality_prompts = ['nq question: who played desmond doss father in hacksaw ridge',
                'nq question: types of skiing in the winter olympics 2018',
                'nq question: where does aarp fall on the political spectrum']

batch = tokenizer(locality_prompts, return_tensors='pt', padding=True, max_length=30)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to(model.device),
    attention_mask=batch['attention_mask'].to(model.device),
    max_new_tokens=15
)
post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to(edited_model.device),
    attention_mask=batch['attention_mask'].to(edited_model.device),
    max_new_tokens=15
)

max_length = batch['input_ids'].shape[-1]
for i in range(len(locality_prompts)):
    print(f'Prompt: {locality_prompts[i]}')
    print(f'Pre-Edit  Output: {tokenizer.decode( pre_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print(f'Post-Edit Output: {tokenizer.decode(post_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print('--'*50 )

Pre-Edit Outputs:  ['</s><s>nq question: who played desmond doss father in hacksaw ridge?\n\nAnswer: The actor who', '<s>nq question: types of skiing in the winter olympics 2018\n\nThere are several types of ski', '</s></s></s></s></s><s>nq question: where does aarp fall on the political spectrum?\n\nAnswer: AARP']
Post-Edit Outputs:  ['</s><s>nq question: who played desmond doss father in hacksaw ridge?\n\nAnswer: The actor who', '<s>nq question: types of skiing in the winter olympics 2018\n\nThere are several types of ski', '</s></s></s></s></s><s>nq question: where does aarp fall on the political spectrum?\n\nAnswer: AARP']


## Citation
If finding this work useful for your research, you can cite it as follows:

```bibtex
@misc{wang2024wiserethinkingknowledgememory,
      title={WISE: Rethinking the Knowledge Memory for Lifelong Model Editing of Large Language Models}, 
      author={Peng Wang and Zexi Li and Ningyu Zhang and Ziwen Xu and Yunzhi Yao and Yong Jiang and Pengjun Xie and Fei Huang and Huajun Chen},
      year={2024},
      eprint={2405.14768},
      archivePrefix={arXiv},
      primaryClass={cs.CL},
      url={https://arxiv.org/abs/2405.14768}, 
}
```